In [10]:
%%python module Camunda

from js import XMLHttpRequest

import json


class Http:
    @staticmethod
    def get(url):
        request = XMLHttpRequest.new()
        request.open("GET", url, False)
        request.send(None)
        assert request.status in [200], request.responseText
        return json.loads(request.responseText or 'null')
        
    @staticmethod
    def post(url, data):
        request = XMLHttpRequest.new()
        request.open("POST", url, False)
        request.setRequestHeader("Content-Type", "application/json")
        request.setRequestHeader("Content-Length", len(json.dumps(data)))
        request.send(json.dumps(data))
        assert request.status in [200, 204], request.responseText
        return json.loads(request.responseText or 'null')

    
class Camunda:

    def __init__(self, worker_id):
        self.base_url = "http://localhost:8080/engine-rest"
        self.worker_id = worker_id

    def get_tasks(self):
        return Http.get(f"{self.base_url}/external-task")
    
    def fetch_and_lock(self, topic):
        payload = {
            "workerId": self.worker_id,
            "maxTasks": 10,
            "topics": [{
                "topicName": topic,
                "lockDuration": 1000 * 60,
            }]
        }
        return Http.post(f"{self.base_url}/external-task/fetchAndLock", payload)
    
    def complete_task(self, task):
        payload = {
            "workerId": self.worker_id,
        }
        return Http.post(f"{self.base_url}/external-task/{task['id']}/complete", payload)   

In [14]:
*** Settings ***

Library  IPython.display
Library  Camunda  worker_id=Robot

In [15]:
*** Tasks ***

Get external tasks
    Get tasks

Log | Report

[
 {
 "activityId" : "Activity_07cyegl" ,
 "activityInstanceId" : "Activity_07cyegl:45e8337f-fb76-11eb-bad0-0242ac110002" ,
 "errorMessage" : null ,
 "executionId" : "45e8337e-fb76-11eb-bad0-0242ac110002" ,
 "id" : "45e83383-fb76-11eb-bad0-0242ac110002" ,
 "lockExpirationTime" : "2021-08-12T14:07:01.546+0000" ,
 "processDefinitionId" : "external-task-demo:1:3216804b-fb76-11eb-bad0-0242ac110002" ,
 "processDefinitionKey" : "external-task-demo" ,
 "processDefinitionVersionTag" : null ,
 "processInstanceId" : "45e7be4c-fb76-11eb-bad0-0242ac110002" ,
 "retries" : null ,
 "suspended" : false ,
 "workerId" : "Robot" ,
 "topicName" : "ExternalTaskTopic" ,
 "tenantId" : null ,
 "priority" : 0 ,
 "businessKey" : null 
 },
 {
 "activityId" : "Activity_07cyegl" ,
 "activityInstanceId" : "Activity_07cyegl:5ac933d8-fb76-11eb-bad0-0242ac110002" ,
 "errorMessage" : null ,
 "executionId" : "5ac933d7-fb76-11eb-bad0-0242ac110002" ,
 "id" : "5ac95aec-fb76-11eb-bad0-0242ac110002" ,
 "lockExpirationTime" : "2021-08-12T14:07:01.547+0000" ,
 "processDefinitionId" : "external-task-demo:1:3216804b-fb76-11eb-bad0-0242ac110002" ,
 "processDefinitionKey" : "external-task-demo" ,
 "processDefinitionVersionTag" : null ,
 "processInstanceId" : "5ac8bea5-fb76-11eb-bad0-0242ac110002" ,
 "retries" : null ,
 "suspended" : false ,
 "workerId" : "Robot" ,
 "topicName" : "ExternalTaskTopic" ,
 "tenantId" : null ,
 "priority" : 0 ,
 "businessKey" : null 
 }
]

In [17]:
*** Tasks ***
    
Complete external tasks
    ${tasks}=  Fetch and lock  ExternalTaskTopic
    FOR  ${task}  IN  @{tasks}
        Display  ${task}
        Complete task  ${task}
    END

Log | Report

[]